<a href="https://colab.research.google.com/github/arjunparmar/TUNEX/blob/main/EmotionDetection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import PIL
import pathlib
import tensorflow as tf

In [2]:
data_dir = "/content/drive/MyDrive/dataset/main"
data_dir = pathlib.Path(data_dir)
img_height = 128
img_width = 128
batch_size = 17640

In [3]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)


14730


In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=5,
  shuffle=True,
  subset="training",
  image_size=(img_height, img_width),
  batch_size=32)

Found 17676 files belonging to 7 classes.
Using 14141 files for training.


In [5]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  seed=5,
  shuffle=True,
  subset="validation",
  image_size=(img_height, img_width),
  batch_size=32)

Found 17676 files belonging to 7 classes.
Using 3535 files for validation.


In [6]:
classes = train_ds.class_names
print(classes)

['0', '1', '2', '3', '4', '5', '6']


In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
model = tf.keras.Sequential([
                             tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
                             tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(128, 128, 3)),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Conv2D(32, (5, 5), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Conv2D(32, (5, 5), activation='relu'),
                             tf.keras.layers.MaxPooling2D((2, 2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation="relu"),
                             tf.keras.layers.Dense(128, activation="relu"),
                             tf.keras.layers.Dense(64, activation="relu"),
                             tf.keras.layers.Dense(32, activation="relu"),
                             tf.keras.layers.Dense(7)
])

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>0.98):
        print("\n 98% acc reached")
        self.model.stop_training = True

In [11]:
model.fit(
  train,
  validation_data=validation,
  epochs=15,
  callbacks=[CustomCallbacks()]
)

Epoch 1/15
442/442 [==============================] - 71s 154ms/step - loss: 1.9450 - accuracy: 0.1432 - val_loss: 1.8290 - val_accuracy: 0.2529
Epoch 2/15
442/442 [==============================] - 7s 16ms/step - loss: 1.6121 - accuracy: 0.3396 - val_loss: 0.9783 - val_accuracy: 0.6068
Epoch 3/15
442/442 [==============================] - 7s 16ms/step - loss: 0.9060 - accuracy: 0.6383 - val_loss: 0.7646 - val_accuracy: 0.7016
Epoch 4/15
442/442 [==============================] - 7s 16ms/step - loss: 0.6086 - accuracy: 0.7640 - val_loss: 0.6147 - val_accuracy: 0.7686
Epoch 5/15
442/442 [==============================] - 7s 16ms/step - loss: 0.4045 - accuracy: 0.8466 - val_loss: 0.4854 - val_accuracy: 0.8385
Epoch 6/15
442/442 [==============================] - 7s 16ms/step - loss: 0.2597 - accuracy: 0.9046 - val_loss: 0.4463 - val_accuracy: 0.8563
Epoch 7/15
442/442 [==============================] - 7s 16ms/step - loss: 0.1939 - accuracy: 0.9311 - val_loss: 0.4180 - val_accuracy: 0.87